In [42]:
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
import matplotlib.pyplot as plt

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [43]:
max_features = 10000  # кількість слів, що розглядаються як особливості
maxlen = 500  # обмеження кількості слів в тексті
batch_size = 32

# Завантаження тренувальних та тестових даних зі вказанням обмеження на кількість слів
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


Як варіант   

from sklearn.model_selection import train_test_split     
розділення тренувального набору на тренувальні та валідаційні   
и
input_train, input_val, y_train, y_val = train_test_split(input_train, y_train, test_size=0.2)

In [44]:
model_rnn = Sequential()
# Додавання Embedding шару для перетворення слів в вектори фіксованої довжини
# Параметри: кількість слів для розгляду як особливості та розмір вектора, що представляє кожне слово
model_rnn.add(Embedding(input_dim=max_features, output_dim=32))

# Додавання SimpleRNN шару з 32 вузлами (кількість нейронів у шарі SimpleRNN)
model_rnn.add(SimpleRNN(32))

# Додавання Dense шару з одним вузлом та активацією sigmoid
model_rnn.add(Dense(1, activation='sigmoid')) # функція активації (sigmoid для бінарної класифікації)

In [45]:
from tensorflow.keras.layers import LSTM

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=max_features, output_dim=32))
#model_lstm.add(LSTM(32, return_sequences=True))  # Перший LSTM шар
model_lstm.add(LSTM(32))                         # Другий LSTM шар
model_lstm.add(Dense(1, activation='sigmoid'))

In [46]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dropout

model_gru = Sequential()
model_gru.add(Embedding(input_dim=max_features, output_dim=32))
model_gru.add(GRU(32))  # 32 - кількість нейронів у шарі GRU
model_gru.add(Dense(64, activation='relu'))  # Додатковий щільний шар
model_gru.add(Dropout(0.5))
model_gru.add(Dense(1, activation='sigmoid'))  # Останній шар для бінарної класифікації

In [47]:
model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_rnn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 32)          320000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 322113 (1.23 MB)
Trainable params: 322113 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
model_lstm.compile(Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), 
                   loss='binary_crossentropy', 
                   metrics=['accuracy'])

model_lstm.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 32)          320000    
                                                                 
 lstm_6 (LSTM)               (None, 32)                8320      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 328353 (1.25 MB)
Trainable params: 328353 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
model_gru.compile(Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

model_gru.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 32)          320000    
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 328513 (1.25 MB)
Trainable params: 328513 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
history = model_rnn.fit(x_train, y_train, epochs=7, batch_size=128, validation_split=0.3)
# Запускаємо навчання моделі на тренувальних даних для 5 епох з розміром пакету 128 та розбиттям на тренувальну та валідаційну підмножини

Epoch 1/10
137/137 [==============================] - 14s 96ms/step - loss: 0.5588 - accuracy: 0.7121 - val_loss: 0.4337 - val_accuracy: 0.8056
Epoch 2/10
137/137 [==============================] - 13s 95ms/step - loss: 0.3705 - accuracy: 0.8517 - val_loss: 0.3696 - val_accuracy: 0.8489
Epoch 3/10
137/137 [==============================] - 14s 99ms/step - loss: 0.2455 - accuracy: 0.9062 - val_loss: 0.3757 - val_accuracy: 0.8447
Epoch 4/10
137/137 [==============================] - 14s 103ms/step - loss: 0.1573 - accuracy: 0.9461 - val_loss: 0.3729 - val_accuracy: 0.8565
Epoch 5/10
137/137 [==============================] - 14s 103ms/step - loss: 0.1172 - accuracy: 0.9610 - val_loss: 0.5095 - val_accuracy: 0.7743
Epoch 6/10
137/137 [==============================] - 14s 101ms/step - loss: 0.0724 - accuracy: 0.9791 - val_loss: 0.4777 - val_accuracy: 0.8299
Epoch 7/10
137/137 [==============================] - 15s 107ms/step - loss: 0.0329 - accuracy: 0.9918 - val_loss: 0.5034 - val_accur

In [54]:
history = model_lstm.fit(x_train, y_train, epochs=7, batch_size=128, validation_split=0.3)

Epoch 1/7
137/137 [==============================] - 30s 206ms/step - loss: 0.3977 - accuracy: 0.8593 - val_loss: 0.3908 - val_accuracy: 0.8515
Epoch 2/7
137/137 [==============================] - 29s 212ms/step - loss: 0.3398 - accuracy: 0.8811 - val_loss: 0.3516 - val_accuracy: 0.8671
Epoch 3/7
137/137 [==============================] - 29s 211ms/step - loss: 0.2949 - accuracy: 0.8976 - val_loss: 0.3278 - val_accuracy: 0.8737
Epoch 4/7
137/137 [==============================] - 28s 203ms/step - loss: 0.2663 - accuracy: 0.9094 - val_loss: 0.3169 - val_accuracy: 0.8743
Epoch 5/7
137/137 [==============================] - 28s 202ms/step - loss: 0.2447 - accuracy: 0.9177 - val_loss: 0.3125 - val_accuracy: 0.8772
Epoch 6/7
137/137 [==============================] - 29s 209ms/step - loss: 0.2234 - accuracy: 0.9255 - val_loss: 0.3015 - val_accuracy: 0.8816
Epoch 7/7
137/137 [==============================] - 28s 206ms/step - loss: 0.2038 - accuracy: 0.9342 - val_loss: 0.3122 - val_accuracy:

In [34]:
history = model_gru.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.3)

Epoch 1/5
137/137 [==============================] - 30s 204ms/step - loss: 0.5639 - accuracy: 0.6895 - val_loss: 0.3592 - val_accuracy: 0.8447
Epoch 2/5
137/137 [==============================] - 28s 205ms/step - loss: 0.2780 - accuracy: 0.8927 - val_loss: 0.4078 - val_accuracy: 0.8161
Epoch 3/5
137/137 [==============================] - 28s 207ms/step - loss: 0.2015 - accuracy: 0.9273 - val_loss: 0.3326 - val_accuracy: 0.8787
Epoch 4/5
137/137 [==============================] - 29s 212ms/step - loss: 0.1485 - accuracy: 0.9494 - val_loss: 0.3697 - val_accuracy: 0.8748
Epoch 5/5
137/137 [==============================] - 30s 220ms/step - loss: 0.1068 - accuracy: 0.9651 - val_loss: 0.4075 - val_accuracy: 0.8727


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Точність на тренуванні')  # Графік точності на тренувальних данних
plt.plot(epochs, val_acc, 'b', label='Точність на валідації')  # Графік точності на валідаційних данних
plt.title('Точність тренування і валідації')  # Заголовок графіку
plt.legend()  # Показати легенду

plt.figure()

plt.plot(epochs, loss, 'bo', label='Втрати на тренуванні')  # Графік втрат на тренувальних данних
plt.plot(epochs, val_loss, 'b', label='Втрати на валідації')  # Графік втрат на валідаційних данних
plt.title('Втрати тренування і валідації')  # Заголовок графіку
plt.legend()  # Показати легенду

plt.show()  # Відобразити графіки

In [ ]:
predictions = model_rnn.predict(x_test)

# Для отримання зрозумілішого вигляду прогнозів, їх можна перетворити в бінарний формат:
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Для порівняння можна вивести деякі прогнозовані значення поряд з фактичними значеннями
for i in range(10):
    print(f"Прогнозоване: {binary_predictions[i]}, Фактичне: {y_test[i]}")

# Додатковий крок: Оцінка моделі
# Це дасть загальну точність моделі на тестових даних
test_loss, test_acc = model_rnn.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [55]:
predictions = model_lstm.predict(x_test)

# Для отримання зрозумілішого вигляду прогнозів, їх можна перетворити в бінарний формат:
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Для порівняння можна вивести деякі прогнозовані значення поряд з фактичними значеннями
for i in range(10):
    print(f"Прогнозоване: {binary_predictions[i]}, Фактичне: {y_test[i]}")

# Додатковий крок: Оцінка моделі
# Це дасть загальну точність моделі на тестових даних
test_loss, test_acc = model_lstm.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

782/782 [==============================] - 26s 33ms/step
Прогнозоване: 0, Фактичне: 0
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 0, Фактичне: 1
Прогнозоване: 0, Фактичне: 0
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 0, Фактичне: 0
Прогнозоване: 1, Фактичне: 0
Прогнозоване: 1, Фактичне: 1
782/782 [==============================] - 26s 33ms/step - loss: 0.3384 - accuracy: 0.8670
Test Loss: 0.3383844792842865
Test Accuracy: 0.8670399785041809


In [40]:
predictions = model_gru.predict(x_test)

# Для отримання зрозумілішого вигляду прогнозів, їх можна перетворити в бінарний формат:
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Для порівняння можна вивести деякі прогнозовані значення поряд з фактичними значеннями
for i in range(10):
    print(f"Прогнозоване: {binary_predictions[i]}, Фактичне: {y_test[i]}")

# Додатковий крок: Оцінка моделі
# Це дасть загальну точність моделі на тестових даних
test_loss, test_acc = model_gru.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

782/782 [==============================] - 50s 63ms/step
Прогнозоване: 0, Фактичне: 0
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 0, Фактичне: 0
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 1, Фактичне: 1
Прогнозоване: 0, Фактичне: 0
Прогнозоване: 1, Фактичне: 0
Прогнозоване: 1, Фактичне: 1
782/782 [==============================] - 37s 46ms/step - loss: 0.4488 - accuracy: 0.8582
Test Loss: 0.4488319158554077
Test Accuracy: 0.858240008354187
